In [1]:
import numpy as np
import pandas as pd
from googleapiclient.discovery import build

In [2]:
# Youtube Data API:
yt = build(
    serviceName = "youtube",
    version = "v3",
    developerKey = "" # my key
)

In [139]:
###### Channels data

# Channels Ids (from content = "https://www.youtube.com/channel/) and titles:
channels = {
    "UCBa659QWEk1AI4Tg--mrJ2A": "Tom Scott",
    "UCXvA7wLxIJDa1SWYWl7e-oQ": "arewethereyet",
    "": ""
}

# Loop in the channels:
df_videos = pd.DataFrame(
    {
        "channel_title": pd.Series([], dtype = "str"),
        "channel_id": pd.Series([], dtype = "str"),
        "video_title": pd.Series([], dtype = "str"),
        "video_id": pd.Series([], dtype = "str"),
        "video_upload_date": pd.Series([], dtype = "str"),
        "views": pd.Series([], dtype = "int"),
        "likes": pd.Series([], dtype = "int"),
        "dislikes": pd.Series([], dtype = "int"),
        "favorites": pd.Series([], dtype = "int"),
        "comments": pd.Series([], dtype = "int")
    }
)
for chnl_ind, chnl_key in enumerate(channels):
    # Choose the channel:
    channel_id = chnl_key
    channel_title = channels[channel_id]

    ### Extract the data from all the videos of the channel

    # Content details:
    content = yt.channels().list(
        id = channel_id,
        part = "contentDetails"
    ).execute()

    # Upload Id:
    upload_id = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # All the channel's videos:
    all_videos = []
    next_pg_token = None
    res = yt.playlistItems().list(
        playlistId = upload_id,
        maxResults = 50,
        part = "snippet",
        pageToken = next_pg_token
    ).execute()
    while True:
        res = yt.playlistItems().list(
            playlistId = upload_id,
            maxResults = 50,
            part = "snippet",
            pageToken = next_pg_token
        ).execute()
        all_videos += res["items"]
        next_pg_token = res.get("nextPageToken")
        if next_pg_token is None:
            break

    # Videos statistics:
    videos_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], all_videos))
    stats = []
    for i in range(0, len(videos_ids), 40):
        res = (yt).videos().list(
            id = ",".join(videos_ids[i:i + 40]),
            part = "statistics"
        ).execute()
        stats += res["items"]

    # Put it in a dataframe:
    video_id, views, likes, dislikes, favorites, comments = [], [], [], [], [], []
    for i in range(len(stats)):
        video_id += [stats[i]["id"]]
        stats_i = stats[i]["statistics"]
        views += [stats_i[k] if "viewCount" in stats_i.keys() else None for k in ["viewCount"]]
        likes += [stats_i[k] if "likeCount" in stats_i.keys() else None for k in ["likeCount"]]
        dislikes += [stats_i[k] if "dislikeCount" in stats_i.keys() else None for k in ["dislikeCount"]]
        favorites += [stats_i[k] if "favoriteCount" in stats_i.keys() else None for k in ["favoriteCount"]]
        comments += [stats_i[k] if "commentCount" in stats_i.keys() else None for k in ["commentCount"]]
    df_channel_videos = pd.DataFrame(
        {
            "channel_title": channel_title,
            "channel_id": channel_id,
            "video_title": list(map(lambda x: x["snippet"]["title"], all_videos)),
            "video_id": video_id,
            "video_upload_date": list(map(lambda x: x["snippet"]["publishedAt"], all_videos)),
            "views": views,
            "likes": likes,
            "dislikes": dislikes,
            "favorites": favorites,
            "comments": comments        
        }
    )
    df_videos = pd.concat([df_videos, df_channel_videos])

In [140]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,favorites,comments
0,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The world's only float-through McDonalds,A6F96xSoLPg,2021-10-08T16:43:23Z,1972534,126298,891,0,4498
1,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,I thought the Schmid Peoplemover was impossible,A2g4u9F9i90,2021-09-30T16:02:45Z,2599438,142280,802,0,5600
2,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The world's most useful model railway,6TLcaJdsRr0,2021-09-24T14:00:54Z,1940761,107008,619,0,3180
3,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The public toll road with no speed limit,10Y-gWNJ2Sw,2021-09-17T08:11:31Z,1933906,128733,813,0,3952
4,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,Three strange river crossings,KVXVNX4Crh4,2021-09-05T13:07:18Z,1467065,87173,428,0,1908
...,...,...,...,...,...,...,...,...,...,...
627,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,Breakfast: Fortified With Iron,MrppkAIVhH4,2006-06-06T22:11:08Z,615806,20732,320,0,2456
0,arewethereyet,UCXvA7wLxIJDa1SWYWl7e-oQ,Around the world with The Tim Traveller Intern...,m4tfntu4XS8,2021-07-12T19:42:09Z,3590,555,1,0,53
1,arewethereyet,UCXvA7wLxIJDa1SWYWl7e-oQ,How the university of Leuven kept losing its l...,N1dEc6e7Smk,2021-05-17T23:48:04Z,900,188,0,0,20
2,arewethereyet,UCXvA7wLxIJDa1SWYWl7e-oQ,The very lively ghost town of Doel.,Q31ChdEFkYk,2021-04-20T23:00:17Z,953,146,0,0,21
